<a href="https://colab.research.google.com/github/DXYoungblood/DataStructures/blob/main/lab/lab04/lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab04.ipynb")

# Lab 4: SQL

In this lab, we are going to practice viewing, sorting, grouping, and merging tables with SQL. We will explore the money donated during the 2016 election using the [Federal Election Commission's public records](https://www.fec.gov/data/). You will be connecting to a `sqlite` database containing the data. The data we will be working with in this lab is quite small (16MB); however, it is a sample taken from a larger database that is more than a few GBs.


### Due Date

This assignment is due at **Tuesday, September 21** at 11:59 pm.

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about this assignment, we ask that you **write your solutions individually**. If you discuss the assignment with others, please **include their names** in the cell below.

**Collaborators:** *list names here*

In [ ]:
import pandas as pd
import sqlalchemy
from ds100_utils import fetch_and_cache
from pathlib import Path

### Connecting to SQLite

Download the database to local disk drive; this will take about a minute.

In [ ]:
db_url = 'https://s3.amazonaws.com/berkeley-ds100/fec_nyc.sqlite'
db_filename = 'fec_nyc.db'
data_dir = Path('data')
fetch_and_cache(db_url, db_filename, data_dir = data_dir)
db_path = data_dir / db_filename

The following cell will connect to the above database using the SQL Alchemy library. This library is more general than the `sqlite3` module from the standard library because it can connect to several different database management systems, such as MySQL and PostgreSQL. It also supports an advanced feature for generating queries called an [object relational mapper](https://docs.sqlalchemy.org/en/latest/orm/tutorial.html) or ORM, which we won't discuss in this course but is quite useful for application development.

In [ ]:
engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = sqlalchemy.inspect(engine)

To check that everything is working we will list the available tables.

In [ ]:
inspector.get_table_names()

## Table Descriptions

Here is a list of the tables in the database.  Each table links to the documentation on the [FEC page](https://www.fec.gov/data/browse-data/) for the dataset.

If you'd like, you can consult the FEC page for the descriptions of the tables to find out what the correspondence is.

- [`cand`](https://www.fec.gov/campaign-finance-data/candidate-summary-file-description/): Candidates table. Contains names and party affiliation.
- [`comm`](https://www.fec.gov/campaign-finance-data/committee-summary-file-description/): Committees table. Contains committee names and types.
- [`indiv_sample_nyc`](https://www.fec.gov/campaign-finance-data/contributions-individuals-file-description/): All individual contributions from New York City.

We can see the schemas of these tables with the following code.

In [ ]:
for exp in connection.execute('SELECT sql FROM sqlite_master;'):
    print(exp[0])

We can also look at the tables themselves.

The individual donations made by NY residents are shown below.

We use `LIMIT 5` to avoid loading a huge table. Although our tables shouldn't get too large to display, this is generally good practice when working in the notebook environment. Jupyter notebooks don't handle very large outputs well. Also note the `"""` to define our query. This allows us to have a query span multiple lines.

In [ ]:
query = """
SELECT *
FROM indiv_sample_nyc
LIMIT 5
"""
pd.read_sql(query, engine)

Similarly, the database of candidates is given below.

In [ ]:
query = """
SELECT *
FROM cand
LIMIT 5
"""
pd.read_sql(query, engine)

And finally, the table of committees is given below.

In [ ]:
query = """
SELECT *
FROM comm
LIMIT 5
"""
pd.read_sql(query, engine)

In CS61A or CS88, we saw basic queries similar to what we see below, which gives only the id and name of candidates from the Democratic party.

In [ ]:
query = """
SELECT cand_id, cand_name
FROM cand
WHERE cand_pty_affiliation = 'DEM'
LIMIT 5
"""
pd.read_sql(query,engine)

---
## Questions

For this lab we will use SQL to extract data out of the `indiv_sample_nyc` table. The schema for this table is given below.

In [ ]:
for exp in connection.execute('SELECT sql FROM sqlite_master WHERE name="indiv_sample_nyc";'):
    print(exp[0])

Let's start by looking at 2016 election contributions made by Donald Trump, who was a NY resident during that year. The following SQL statement returns the `cmte_id`, `transaction_amt`, and `name` for every contribution made by any donor with "DONALD" and "TRUMP" in their name.

Notice the `WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%'` clause.

In [ ]:
query1 = """
SELECT 
    cmte_id,
    transaction_amt,
    name
FROM indiv_sample_nyc
WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%'
"""

res = pd.read_sql(query1, engine)
res

### Question 1

If we look at the list above, it appears that some donations were not by Donald Trump himself, but instead by an entity called "DONALD J TRUMP FOR PRESIDENT INC". Fortunately, we see that our query only seems to have picked up one such anomalous name.

In [ ]:
res['name'].value_counts()

Revise the above query so that the 15 donations made by "DONALD J TRUMP FOR PRESIDENT INC" do not appear. Your resulting table should have 142 rows. 

Hint: Copy the above query into the cell below as a starting point!

Hint: Consider using `NOT LIKE`.

<!--
BEGIN QUESTION
name: q1
-->

In [ ]:
query_q1 = """
SELECT
    ... 
    ... 
    ...
FROM indiv_sample_nyc
WHERE ...
"""


res_q1 = pd.read_sql(query_q1, engine)
res_q1

In [ ]:
grader.check("q1")

### Question 2

When you printed the schema for the tables, you may have noticed that both the `cand` and `comm` tables have a `cand_id` column. Let's try joining these two tables on this column.

List the first 5 candidate names (`cand_name`) in reverse lexicographic order by `cand_name`, along with their corresponding committee names. Only select rows that have a matching `cand_id` in both tables.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cand_name</th>
      <th>cmte_nm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ZUTLER, DANIEL PAUL MR</td>
      <td>CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ZUMWALT, JAMES</td>
      <td>ZUMWALT FOR CONGRESS</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ZUKOWSKI, ANDREW GEORGE</td>
      <td>ZUKOWSKI FOR CONGRESS</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ZUCCOLO, JOE</td>
      <td>JOE ZUCCOLO FOR CONGRESS</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ZORN, ROBERT ERWIN</td>
      <td>CONSTITUTIONAL COMMITTEE</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2
-->

In [ ]:
query_q2 = """
SELECT ..., ...
FROM cand c1 ... JOIN comm c2 ON ...
...
...
"""


res_q2 = pd.read_sql(query_q2, engine)
res_q2

In [ ]:
grader.check("q2")

### Question 3

As in the previous part, list the first 5 candidate names (`cand_name`) by reverse lexicographic order and their corresponding committee names (`cmte_nm`). This time, include all candidate names. If there is no matching `cand_id` in the `comm` table, then `cmte_nm` should be NULL (or `None` in the Python representation).

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cand_name</th>
      <th>cmte_nm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ZUTLER, DANIEL PAUL MR</td>
      <td>CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ZUMWALT, JAMES</td>
      <td>ZUMWALT FOR CONGRESS</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ZUKOWSKI, ANDREW GEORGE</td>
      <td>ZUKOWSKI FOR CONGRESS</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ZUCCOLO, JOE</td>
      <td>JOE ZUCCOLO FOR CONGRESS</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ZORNOW, TODD MR</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q3
-->

In [ ]:
query_q3 = """
SELECT ..., ...
FROM cand c1 ... JOIN comm c2 ON ...
...
...
"""


res_q3 = pd.read_sql(query_q3, engine)
res_q3

In [ ]:
grader.check("q3")

### Question 4

As a review of some of the key ideas from CS61A/CS88, let's refine our query above.

We see that many of the contributions above were to the same committee. Create a new SQL query that returns the total amount that Donald Trump contributed to each committee. Your table should have four columns: `cmte_id`, `total_amount`, `num_donations`, and `cmte_nm`. The `total_amount` is the total amount contributed to that committee, `num_donations` is the total number of donations, and `cmte_nm` is the name of the committee. Note that `cmte_nm` is not available in `indiv_sample_nyc`, so you will have to somehow obtain information from the `comm` table.

Your final table should be sorted in decreased order of `total_amount`.

**Note**: This is a hard question! Don't be afraid to reference the walkthrough video if you're stuck.

Hint: Use `GROUP BY` as covered in CS61A and CS88.

Hint: Use `COUNT(*)` as covered in CS61A and CS88.

Hint: Use `SUM()` as covered in CS61A and CS88.

Hint: Use `ORDER BY` as covered in CS61A and CS88.

Hint: Use `DESC` as covered in CS61A and CS88.

Hint: Your `WHERE` clause should be the same as question 1a.

Hint: You'll probably need either a join, or a subquery.

<!--
BEGIN QUESTION
name: q4
-->

In [ ]:
query_q4 = """
...
"""


res_q4 = pd.read_sql(query_q4, engine)
res_q4

In [ ]:
grader.check("q4")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)